# Первичный анализ рынка книг

## Задача 

Проанализировать базу данных и найти ценную информацию, ответив на следующие поставленные вопросы:
- Сколько книг вышло после 1 января 2000 года?
- Количество обзоров и средняя оценка для каждой книги?
- Издательство, которое выпустило наибольшее число книг толще 50 страниц? (так мы исключим из анализа брошюры)
- Автора с самой высокой средней оценкой книг? (50 и более оценок)
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок?

## Подключение к базе данных

In [2]:
#импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Выполнение запросов

Для начала изучим каждую из таблиц

In [21]:
query = '''
SELECT
    *
FROM
    books
'''
print('Таблица BOOKS')
pd.io.sql.read_sql(query, con = engine).head()

Таблица BOOKS


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [22]:
query = '''
SELECT
    *
FROM
    authors
'''
print('Таблица AUTHORS')
pd.io.sql.read_sql(query, con = engine).head()

Таблица AUTHORS


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [23]:
query = '''
SELECT
    *
FROM
    ratings
'''
print('Таблица RATINGS')
pd.io.sql.read_sql(query, con = engine).head()

Таблица RATINGS


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [24]:
query = '''
SELECT
    *
FROM
    reviews
'''
print('Таблица REVIEWS')
pd.io.sql.read_sql(query, con = engine).head()

Таблица REVIEWS


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [25]:
query = '''
SELECT
    *
FROM
    publishers
'''
print('Таблица PUBLISHERS')
pd.io.sql.read_sql(query, con = engine).head()

Таблица PUBLISHERS


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Самая интересная таблица - *books* - она связывает оставшиеся четыре таблицы между собой. С ней и начнем работу

### Сколько книг вышло после 1 января 2000 года?

Для начала найдем общее количество более-менее современных книг (вышедших после 1-го января 2000-го года)

In [12]:
query1 = '''
SELECT
    COUNT (book_id) as cnt
FROM
    books
WHERE
    publication_date > '2000-01-01'
'''
pd.io.sql.read_sql(query1, con = engine)

,cnt
0,819


In [15]:
query11 = '''
SELECT
    MAX(publication_date) as max_date
FROM
    books
'''
pd.io.sql.read_sql(query11, con = engine)

,max_date
0,2020-03-31


819 книг за 20 лет - не так уж и много - всего по 40 книг в год

### Количество обзоров и средняя оценка для каждой книги?

Теперь узнаем лидеров среди книг по числу обзоров и найдем для них среднюю оценку

In [31]:
query2 = '''
SELECT
    title,
    AVG(rating) as avg_rating,
    COUNT(review_id) as cnt_reviews
FROM
    books
    LEFT JOIN ratings ON books.book_id = ratings.book_id
    LEFT JOIN reviews ON books.book_id = reviews.book_id
GROUP BY
    title
ORDER BY
    cnt_reviews DESC
'''
pd.io.sql.read_sql(query2, con = engine).head(10)

,title,avg_rating,cnt_reviews
0,Twilight (Twilight #1),3.662500,1120
1,The Hobbit or There and Back Again,4.125000,528
2,The Catcher in the Rye,3.825581,516
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,492
4,Harry Potter and the Chamber of Secrets (Harry...,4.287500,480
5,Angels & Demons (Robert Langdon #1),3.678571,420
6,Harry Potter and the Order of the Phoenix (Har...,4.186667,375
7,The Lightning Thief (Percy Jackson and the Oly...,4.080645,372
8,The Fellowship of the Ring (The Lord of the Ri...,4.391892,370
9,Animal Farm,3.729730,370


Как мы видим, Сумерки заметно опережают преследователей, хотя по рейтингу даже не добрались до "четверки"

### Издательство, которое выпустило наибольшее число книг толще 50 страниц?

In [34]:
query3 = '''
SELECT
    publisher,
    COUNT(book_id) as cnt_books
FROM
    books
    LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE
    num_pages > 50
GROUP BY
    publisher
ORDER BY
    cnt_books DESC
'''
pd.io.sql.read_sql(query3, con = engine).head(10)

,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,Delta,13


Перед нами десятка самых печатающих издательств. Пингвины впереди всех!

### Автора с самой высокой средней оценкой книг?

In [37]:
query4 = '''
SELECT
    author,
    AVG(rating) as avg_rating
FROM
    books
    LEFT JOIN authors ON books.author_id = authors.author_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE
    num_pages > 50    
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''
pd.io.sql.read_sql(query4, con = engine).head(10)

,author,avg_rating
0,Charles Dickens,5.0
1,Jon Kabat-Zinn,5.0
2,James Patterson/مریم کاظمی‌تبار,5.0
3,William Faulkner,5.0
4,George Orwell/Lionel Trilling,5.0
5,Pat Frank,5.0
6,Kerry Patterson/Joseph Grenny/Ron McMillan/Al ...,5.0
7,Michael Lewis,5.0
8,Chaim Potok,5.0
9,Robert Kirkman/Tony Moore/Charlie Adlard/Cliff...,5.0


Ровно 5 баллов - это либо идеальные писатели, либо малое количество оценок. Добавим в запрос показатель количества оценок и посмотрим, сколько их у лидеров по среднему рейтингу

In [46]:
query4 = '''
SELECT
    author,
    AVG(rating) as avg_rating,
    COUNT(rating) as cnt_rating
FROM
    books
    LEFT JOIN authors ON books.author_id = authors.author_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE
    num_pages > 50    
GROUP BY
    author
ORDER BY
    avg_rating DESC, cnt_rating DESC
'''
pd.io.sql.read_sql(query4, con = engine).head(10)

,author,avg_rating,cnt_rating
0,Michael Lewis,5.0,3
1,James Clavell,5.0,2
2,Carl Sagan/Ann Druyan,5.0,2
3,Vince Flynn,5.0,2
4,James Patterson/مریم کاظمی‌تبار,5.0,2
5,William Faulkner,5.0,2
6,Robert Kirkman/Tony Moore/Charlie Adlard/Cliff...,5.0,2
7,Marguerite Henry/Wesley Dennis,5.0,2
8,Kerry Patterson/Joseph Grenny/Ron McMillan/Al ...,5.0,2
9,Katherine Dunn,5.0,2


Как и ожидалось, у всех, кто имеет "чистую пятёрку" оценок не больше, чем 3. Вряд ли это можно считать показателем успешности автора. Теперь отсортируем по числу оценок

In [47]:
query4 = '''
SELECT
    author,
    AVG(rating) as avg_rating,
    COUNT(rating) as cnt_rating
FROM
    books
    LEFT JOIN authors ON books.author_id = authors.author_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE
    num_pages > 50    
GROUP BY
    author
ORDER BY
    cnt_rating DESC, avg_rating DESC
'''
pd.io.sql.read_sql(query4, con = engine).head(10)

,author,avg_rating,cnt_rating
0,J.K. Rowling/Mary GrandPré,4.288462,312
1,J.R.R. Tolkien,4.240964,166
2,Stephenie Meyer,3.662500,160
3,Dan Brown,3.741259,143
4,Nicholas Sparks,3.882883,111
5,Stephen King,4.009434,106
6,J.D. Salinger,3.846939,98
7,Rick Riordan,4.130952,84
8,Jodi Picoult,3.881579,76
9,George Orwell/Boris Grabnar/Peter Škerl,3.729730,74


Другое дело! Сразу видно, кто в топе по популярности, пусть и средний рейтинг у данных писателей ниже

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок?

In [66]:
query5 = '''
SELECT
    AVG(cnts_reviews.cnt_reviews) AS avg_cnt_reviews
FROM
    (SELECT
        COUNT(review_id) AS cnt_reviews
    FROM (
        SELECT
            username
        FROM
            ratings
        GROUP BY
            username
        HAVING
            COUNT(rating_id) > 50
            ) AS users_with_50_plus_ratings
        LEFT JOIN reviews ON users_with_50_plus_ratings.username = reviews.username
    GROUP BY
        reviews.username) AS cnts_reviews
'''
pd.io.sql.read_sql(query5, con = engine)

,avg_cnt_reviews
0,24.333333


Количество отзывов даже у самых активных пользователей почти в 2 раза меньше, чем количество оценок. Не пора ли простимулировать пользователей писать больше отзывов?